## Testes et regregression logistique

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler

file = open('dataset/dataset.pkl','rb')
DATA = pickle.load(file)
pheno = DATA['pheno']
X_gpa = DATA['X_gpa']
X_snps = DATA['X_snps']
X_genexp = DATA['X_genexp']



In [2]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, make_scorer
from sklearn.feature_selection import chi2
from scipy.stats import f_oneway
from statsmodels.stats.multitest import multipletests
import pandas as pd
import numpy as np

# Fonction pour le CHI2 avec correction FDR BH
def chi2_fdr_selection(X, y, alpha=0.05):
    col_names = [i for i in range(X.shape[1])]
    X = pd.DataFrame(X, columns=col_names)

    _, p_values = chi2(X, y)

    # Correction méthode Benjamini-Hochberg
    _, p_values_corrected, _, _ = multipletests(p_values, method='fdr_bh')

    selected_columns = set(X.columns[p_values_corrected < alpha])

    return selected_columns

# Fonction pour l'ANOVA avec correction FDR BH
def anova_fdr_selection(X, y, alpha=0.05):
    col_names = [i for i in range(X.shape[1])]
    X = pd.DataFrame(X, columns=col_names)

    p_values = []
    for col in X.columns:
        groups = [X[col][y == label] for label in pd.unique(y)]
        _, p_value = f_oneway(*groups)
        p_values.append(p_value)

    _, p_values_corrected, _, _ = multipletests(p_values, method='fdr_bh')

    selected_columns = set(X.columns[p_values_corrected < alpha])

    return selected_columns

# Fonction principale

def cross_validate_model(pheno, X_gpa, X_snps, X_genexp, alpha=0.05):
    performances = []

    # Validation croisée à 5 plis
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for antibiotique in pheno.columns[1:]:
        print(f"Traitement de l'antibiotique : {antibiotique}")

        # Extraire la cible y
        y = pheno[antibiotique].to_numpy()
        valid_indices = ~np.isnan(y)
        y = y[valid_indices]

        X_gpa_filtered = X_gpa[valid_indices]
        X_snps_filtered = X_snps[valid_indices]
        X_genexp_filtered = X_genexp[valid_indices]

        recall_scores = []
        selected_columns_gpa = []
        selected_columns_snps = []
        selected_columns_genexp = []

        for train_idx, test_idx in skf.split(X_gpa_filtered, y):
            # Diviser les données
            X_train_gpa, X_test_gpa = X_gpa_filtered[train_idx], X_gpa_filtered[test_idx]
            X_train_snps, X_test_snps = X_snps_filtered[train_idx], X_snps_filtered[test_idx]
            X_train_genexp, X_test_genexp = X_genexp_filtered[train_idx], X_genexp_filtered[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Standardisation des données
            scaler = StandardScaler()
            X_train_genexp_scaled = scaler.fit_transform(X_train_genexp)
            X_test_genexp_scaled = scaler.transform(X_test_genexp)

            # Sélection de caractéristiques
            selected_gpa = list(chi2_fdr_selection(X_train_gpa, y_train, alpha))
            selected_snps = list(chi2_fdr_selection(X_train_snps, y_train, alpha))
            selected_genexp = list(anova_fdr_selection(X_train_genexp_scaled, y_train, alpha))

            selected_columns_gpa.extend(selected_gpa)
            selected_columns_snps.extend(selected_snps)
            selected_columns_genexp.extend(selected_genexp)

            # Filtrer les caractéristiques sélectionnées
            X_train_gpa_filtered = X_train_gpa[:, selected_gpa]
            X_test_gpa_filtered = X_test_gpa[:, selected_gpa]
            X_train_snps_filtered = X_train_snps[:, selected_snps]
            X_test_snps_filtered = X_test_snps[:, selected_snps]
            X_train_genexp_filtered = X_train_genexp_scaled[:, selected_genexp]
            X_test_genexp_filtered = X_test_genexp_scaled[:, selected_genexp]

            # Combiner les caractéristiques
            X_train_combined = np.hstack([
                X_train_gpa_filtered, X_train_snps_filtered, X_train_genexp_filtered
            ])
            X_test_combined = np.hstack([
                X_test_gpa_filtered, X_test_snps_filtered, X_test_genexp_filtered
            ])

            # Modèle de régression logistique
            model = LogisticRegression(max_iter=500)
            model.fit(X_train_combined, y_train)

            # Prédiction et évaluation
            y_pred = model.predict(X_test_combined)
            recall_macro = recall_score(y_test, y_pred, average='macro')
            recall_scores.append(recall_macro)

        # Résultats
        mean_recall = np.mean(recall_scores)
        selected_columns_gpa = list(set(selected_columns_gpa))
        selected_columns_snps = list(set(selected_columns_snps))
        selected_columns_genexp = list(set(selected_columns_genexp))

        print(f"Rappel moyen pour {antibiotique} : {mean_recall:.4f}")

        performances.append({
            'Antibiotique': antibiotique,
            'Recall': mean_recall,
            'Colonnes_GPA': selected_columns_gpa,
            'Colonnes_SNPs': selected_columns_snps,
            'Colonnes_Expression_Genetique': selected_columns_genexp
        })

    performances_df = pd.DataFrame(performances)

    # Sauvegarder les performances dans un fichier CSV
    performances_df.to_csv('result_crossval.csv', index=False, sep=';')
    print("Les performances ont été enregistrées dans 'result_crossval.csv'.")

    return performances_df


In [3]:

performances_df = cross_validate_model(pheno, X_gpa, X_snps, X_genexp, alpha=0.05)

print("\nPerformances de la régression logistique :")
print(performances_df)

Traitement de l'antibiotique : Tobramycin
Rappel moyen pour Tobramycin : 0.8917
Traitement de l'antibiotique : Ceftazidim
Rappel moyen pour Ceftazidim : 0.7480
Traitement de l'antibiotique : Ciprofloxacin
Rappel moyen pour Ciprofloxacin : 0.7972
Traitement de l'antibiotique : Meropenem
Rappel moyen pour Meropenem : 0.8243
Traitement de l'antibiotique : Colistin
Rappel moyen pour Colistin : 0.6781
Les performances ont été enregistrées dans 'result_crossval.csv'.

Performances de la régression logistique :
    Antibiotique    Recall Colonnes_GPA Colonnes_SNPs  \
0     Tobramycin  0.891733           []            []   
1     Ceftazidim  0.748039           []            []   
2  Ciprofloxacin  0.797229           []            []   
3      Meropenem  0.824269           []            []   
4       Colistin  0.678090           []            []   

                       Colonnes_Expression_Genetique  
0  [1, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 19, 2...  
1  [4098, 4118, 2072, 2074, 28, 32, 3